<a href="https://colab.research.google.com/github/informatics-isi-edu/eye-ai-exec/blob/main/notebooks/VGG19_Diagnosis_Predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# VGG19 Model Application

This notebook applied a pre-trained model to a dataset specified in the configuration file and uploads the labels to the catalog.  The ROC curve is also calculated and uploaded.


In [ ]:
# Prerequisites to configure colab
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install deriva
    !pip install bdbag
    !pip install --upgrade --force pydantic
    !pip install git+https://github.com/informatics-isi-edu/deriva-ml git+https://github.com/informatics-isi-edu/eye-ai-ml
    !pip install setuptools_git_versioning


In [ ]:
repo_dir = "Repos"   # Set this to be where your github repos are located.
%load_ext autoreload
%autoreload 2

# Update the load path so python can find modules for the model
import sys
from pathlib import Path
sys.path.insert(0, str(Path.home() / repo_dir / "eye-ai-ml"))

In [ ]:
# Prerequisites

import json
import os
from eye_ai.eye_ai import EyeAI
import pandas as pd
from pathlib import Path, PurePath
import logging
# import torch

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

In [ ]:

from deriva.core.utils.globus_auth_utils import GlobusNativeLogin
catalog_id = "eye-ai" #@param
host = 'www.eye-ai.org'


gnl = GlobusNativeLogin(host=host)
if gnl.is_logged_in([host]):
    print("You are already logged in.")
else:
    gnl.login([host], no_local_server=True, no_browser=True, refresh_tokens=True, update_bdbag_keychain=True)
    print("Login Successful")

Connect to Eye-AI catalog.  Configure to store data local cache and working directories.  Initialize Eye-AI for pending execution based on the provided configuration file.

In [ ]:
# Variables to configure the rest of the notebook.

cache_dir = '/data'        # Directory in which to cache materialized BDBags for datasets
working_dir = '/data'    # Directory in which to place output files for later upload.

configuration_rid="2-C9F2"      # Configuration file for this run.  Needs to be changed for each execution.

In [ ]:
EA = EyeAI(hostname = host, catalog_id = catalog_id, cache_dir= cache_dir, working_dir=working_dir)

In [ ]:
# @title Initiate an Execution
configuration_records = EA.execution_init(configuration_rid=configuration_rid)
input_dataset = configuration_records.bag_paths[0] # Assumes that the configuration file only specifies one dataset.
configuration_records.model_dump()

# {'caching_dir': PosixPath('/data'),
#  'working_dir': PosixPath('/data/sreenidhi/EyeAI_working'),
#  'vocabs': {'Workflow_Type': [{'name': 'VGG19_Catalog_Model_Prediction',
#     'rid': '2-C9AT'}],
#   'Execution_Asset_Type': [{'name': 'VGG19_Catalog_Model_Prediction',
#     'rid': '2-C9AW'}]},
#  'execution_rid': '2-C9B6',
#  'workflow_rid': '2-C9AY',
#  'bag_paths': [PosixPath('/data/2-277M_8c4b855c2752e098580a5bb0d1b63a8cedde4462805fe74cddc912a72fb39963/Dataset_2-277M')],
#  'assets_paths': [PosixPath('/data/sreenidhi/EyeAI_working/Execution_Assets/VGG19_Catalog_LAC_DHS_Cropped_Data_exlcuding_no_Optic_disc_fundus_Trained_model_June_24_2024.h5'),
#   PosixPath('/data/sreenidhi/EyeAI_working/Execution_Assets/best_hyperparameters_exluding_no_optic_disc_images_june_24_2024.json')],
#  'configuration_path': PosixPath('/data/sreenidhi/EyeAI_working/Execution_Metadata/Execution_Config-vgg19_catalog_model_prediction_more_metrics_sreenidhi_june_24_2024.json')}

Algorithm was trained on cropped images, so take the raw images and bounding boxes and apply, storing the results in the working directory.

In [ ]:
configuration_records

In [ ]:
str(EA.working_dir)

In [ ]:
# @title Get Cropped Images
cropped_image_path, cropped_csv = EA.create_cropped_images(str(configuration_records.bag_paths[0]),
                                                           output_dir = str(EA.working_dir),
                                                           crop_to_eye=True)

In [ ]:
import os

def count_files(directory):
    return len([name for name in os.listdir(directory) if os.path.isfile(os.path.join(directory, name))])

def analyze_graded_test_dataset(base_path):
    main_folders = ['Image_cropped'] #, 
    
    for main_folder in main_folders:
        main_folder_path = os.path.join(base_path, main_folder)
        if not os.path.exists(main_folder_path):
            print(f"{main_folder} folder not found")
            continue
        
        print(f"\nAnalyzing {main_folder} folder:")
        
        total_files = 0
        for angle_folder in os.listdir(main_folder_path):
            angle_folder_path = os.path.join(main_folder_path, angle_folder)
            if os.path.isdir(angle_folder_path):
                file_count = count_files(angle_folder_path)
                print(f"  {angle_folder}: {file_count} images")
                total_files += file_count
        
        print(f"Total images in {main_folder}: {total_files}")

# Usage
base_path = "/data/sreenidhi/EyeAI_working/"
analyze_graded_test_dataset(base_path)


In [ ]:
best_hyper_parameters_json_path = str(configuration_records.assets_paths[1])
best_hyper_parameters_json_path

In [ ]:
model_path = configuration_records.assets_paths[0]
model_path 

In [ ]:
cropped_image_path

In [ ]:

output_path = str(EA.working_dir) + "/Execution_Assets/" + configuration_records.vocabs['Execution_Asset_Type'][0].name
os.mkdir(output_path)

output_path

Import the actual model code and then run against the input dataset specified in the configuration file.  

In [ ]:
# @title Execute Proecss algorithm (Test model)
from eye_ai.models.vgg19_diagnosis_predict_more_metrics import prediction

with EA.execution(execution_rid=configuration_records.execution_rid) as exec:
  prediction(
      model_path = model_path, 
      cropped_image_path = cropped_image_path, 
      output_dir = output_path, 
      best_hyperparameters_json_path = best_hyper_parameters_json_path)

In [ ]:
# @title Save Execution Assets (model) and Metadata
uploaded_assets = EA.execution_upload(configuration_records.execution_rid, True)
